# 🏋️‍♂️ AI FITNESS COACH 

## Sistema de Geração de Planilhas de Treino com LangGraph

**Trabalho Prático 03 - LangGraph**  
**Disciplina:** Redes Neurais Artificiais  
**Objetivo:** Desenvolver um agente de IA capaz de gerar planos de treino de musculação personalizados usando LangGraph

---

### 📋 **Funcionalidades do Sistema:**

1. **Coleta e Validação de Dados** - Idade, peso, periodicidade e objetivo
2. **Busca Web Inteligente** - Pesquisa as melhores práticas de treino
3. **Cálculos Matemáticos** - IMC, gasto calórico e distribuição de treinos
4. **Sistema RAG** - Base de conhecimento fitness vetorizada
5. **Geração de Conteúdo** - 12 treinos completos e personalizados

### 🏗️ **Arquitetura LangGraph:**

```
[Entrada] → [Validação] → [Web Search] → [Cálculos] → [RAG] → [Geração] → [Saída]
```

---

In [4]:
# 📦 INSTALAÇÃO E IMPORTAÇÃO DE BIBLIOTECAS

# Primeiro, vamos verificar se todas as bibliotecas estão instaladas
import sys
import subprocess
import importlib

def verificar_instalacao(pacote):
    """Verifica se um pacote está instalado"""
    try:
        importlib.import_module(pacote)
        return True
    except ImportError:
        return False

# Mapeamento correto: nome_display → nome_real_do_módulo
pacotes_verificacao = {
    'langchain': 'langchain',
    'langgraph': 'langgraph', 
    'langchain_community': 'langchain_community',
    'langchain_groq': 'langchain_groq',
    'langchain_chroma': 'langchain_chroma',
    'duckduckgo_search': 'duckduckgo_search',
    'sentence_transformers': 'sentence_transformers',
    'python_dotenv': 'dotenv',  # ← Este é o mapeamento correto!
    'groq': 'groq'
}

print("🔍 Verificando instalações...")
todos_ok = True
for nome_display, nome_modulo in pacotes_verificacao.items():
    status = verificar_instalacao(nome_modulo)
    emoji = "✅" if status else "❌"
    print(f"{emoji} {nome_display}")
    if not status:
        todos_ok = False

if todos_ok:
    print("\n🎉 Todos os pacotes estão instalados e funcionando!")
else:
    print("\n💡 Se algum pacote estiver com ❌, execute: pip install -r requirements.txt")
    
print("\n📋 Status geral:", "✅ PRONTO" if todos_ok else "⚠️ VERIFICAR INSTALAÇÃO")

🔍 Verificando instalações...
✅ langchain
✅ langgraph
✅ langchain_community
✅ langchain_groq
✅ langchain_chroma
✅ duckduckgo_search
✅ sentence_transformers
✅ python_dotenv
✅ groq

🎉 Todos os pacotes estão instalados e funcionando!

📋 Status geral: ✅ PRONTO


In [ ]:
# 🔧 IMPORTAÇÕES PRINCIPAIS

import os
import math
import re
from typing import Dict, Any, List, Optional, Tuple
from pathlib import Path
from datetime import datetime
import json
import numpy as np
import hashlib

# LangGraph (importações básicas)
from typing_extensions import TypedDict

# Tentar importar LangGraph
try:
    from langgraph.graph import StateGraph, END
    LANGGRAPH_DISPONIVEL = True
    print("✅ LangGraph disponível")
except ImportError:
    print("⚠️ LangGraph não disponível - usando implementação alternativa")
    LANGGRAPH_DISPONIVEL = False
    # Implementação alternativa simples
    class StateGraph:
        def __init__(self, state_class):
            self.nodes = {}
            self.entry_point = None
            
        def add_node(self, name, func):
            self.nodes[name] = func
            
        def set_entry_point(self, name):
            self.entry_point = name
            
        def add_edge(self, from_node, to_node):
            pass  # Simplificado
            
        def compile(self):
            return SimpleWorkflow(self.nodes, self.entry_point)
    
    END = "END"
    
    class SimpleWorkflow:
        def __init__(self, nodes, entry_point):
            self.nodes = nodes
            self.entry_point = entry_point
            
        def invoke(self, state):
            # Execução sequencial simplificada
            current = state
            for node_name in ["validacao", "calculos", "busca_web", "rag", "geracao"]:
                if node_name in self.nodes:
                    try:
                        current = self.nodes[node_name](current)
                    except Exception as e:
                        print(f"Erro no nó {node_name}: {e}")
                        break
            return current

# Tentar importar ferramentas externas
try:
    from duckduckgo_search import DDGS
    BUSCA_WEB_DISPONIVEL = True
except ImportError:
    print("⚠️ duckduckgo_search não disponível - busca web desabilitada")
    BUSCA_WEB_DISPONIVEL = False
    DDGS = None

try:
    from sentence_transformers import SentenceTransformer
    EMBEDDINGS_DISPONIVEL = True
except ImportError:
    print("⚠️ sentence_transformers não disponível - usando busca simples")
    EMBEDDINGS_DISPONIVEL = False
    SentenceTransformer = None

# Configuração de ambiente
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    print("⚠️ python-dotenv não disponível")

# Configurações de display
import warnings
warnings.filterwarnings('ignore')

print("✅ Todas as importações realizadas com sucesso!")
print(f"📅 Data/Hora: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
print(f"🔧 LangGraph: {'✅' if LANGGRAPH_DISPONIVEL else '⚠️ (alternativa)'}")
print(f"🌐 Busca Web: {'✅' if BUSCA_WEB_DISPONIVEL else '❌'}")
print(f"🧠 Embeddings: {'✅' if EMBEDDINGS_DISPONIVEL else '❌'}")

✅ Todas as importações realizadas com sucesso!
📅 Data/Hora: 04/10/2025 22:56:45


---

## 🏗️ **1. DEFINIÇÃO DO ESTADO DO GRAFO**

O estado do LangGraph manterá todas as informações que fluem entre os nós:

- **Dados do usuário** (idade, peso, periodicidade, objetivo)
- **Resultados de validação** 
- **Informações da busca web**
- **Cálculos matemáticos** (IMC, calorias)
- **Conhecimento RAG** recuperado
- **Planilha final** gerada

In [26]:
# 📊 DEFINIÇÃO DO ESTADO DO LANGGRAPH

class FitnessCoachState(TypedDict):
    """Estado que flui através dos nós do LangGraph"""
    
    # Entrada do usuário
    dados_usuario: Dict[str, Any]
    
    # Validação
    validacao_ok: bool
    erros_validacao: List[str]
    dados_limpos: Dict[str, Any]
    
    # Cálculos matemáticos
    imc: float
    classificacao_imc: str
    gasto_calorico: int
    distribuicao_treinos: Dict[str, Any]
    
    # Busca web
    informacoes_web: List[Dict[str, str]]
    
    # Sistema RAG Avançado
    conhecimento_rag: List[str]
    exercicios_sugeridos: Dict[str, List[str]]
    principios_treino: Dict[str, Any]
    contexto_rag_completo: str  # ← Novo: contexto RAG completo
    insights_rag: List[str]     # ← Novo: insights específicos do RAG
    
    # Geração final
    planilha_treinos: List[Dict[str, Any]]
    resumo_final: str
    
    # Controle de fluxo
    proximo_no: str
    mensagens: List[str]

print("✅ Estado do LangGraph atualizado!")
print("📝 Novos campos RAG adicionados:")
print("   • contexto_rag_completo: Contexto semântico completo")
print("   • insights_rag: Insights específicos extraídos")
print("📊 Total de campos: 18 (incluindo 2 novos campos RAG)")

✅ Estado do LangGraph atualizado!
📝 Novos campos RAG adicionados:
   • contexto_rag_completo: Contexto semântico completo
   • insights_rag: Insights específicos extraídos
📊 Total de campos: 18 (incluindo 2 novos campos RAG)


---

## 🔧 **2. FERRAMENTAS EXTERNAS**

Vamos criar as ferramentas que serão usadas pelos nós do LangGraph:

In [27]:
# 🧮 CALCULADORA FITNESS

class FitnessCalculator:
    """Calculadora para métricas fitness importantes"""
    
    @staticmethod
    def calcular_imc(peso: float, altura: float) -> Tuple[float, str]:
        """Calcula IMC e classificação"""
        imc = peso / (altura ** 2)
        
        if imc < 18.5:
            classificacao = "Abaixo do peso"
        elif 18.5 <= imc < 25:
            classificacao = "Peso normal"
        elif 25 <= imc < 30:
            classificacao = "Sobrepeso"
        else:
            classificacao = "Obesidade"
            
        return round(imc, 2), classificacao
    
    @staticmethod
    def estimar_gasto_calorico(peso: float, objetivo: str, duracao: int = 60) -> int:
        """Estima gasto calórico durante treino"""
        multiplicadores = {
            "hipertrofia": 5.5,
            "forca": 4.5,
            "emagrecimento": 6.5,
            "condicionamento": 7.0
        }
        
        multiplicador = multiplicadores.get(objetivo.lower(), 5.5)
        calorias_por_minuto = (peso * multiplicador) / 60
        return int(calorias_por_minuto * duracao)
    
    @staticmethod
    def calcular_distribuicao_treinos(periodicidade: int) -> Dict[str, Any]:
        """Calcula distribuição de 12 treinos"""
        distribuicoes = {
            2: {"tipo": "Full Body", "ciclos": 6, "grupos_por_treino": "Corpo todo"},
            3: {"tipo": "ABC", "ciclos": 4, "grupos_por_treino": "2-3 grupos"},
            4: {"tipo": "ABCD", "ciclos": 3, "grupos_por_treino": "1-2 grupos"},
            5: {"tipo": "ABCDE", "ciclos": 2.4, "grupos_por_treino": "1-2 grupos"},
            6: {"tipo": "Push/Pull/Legs 2x", "ciclos": 2, "grupos_por_treino": "Específicos"}
        }
        
        return distribuicoes.get(periodicidade, distribuicoes[4])

# Teste da calculadora
calc = FitnessCalculator()
imc_teste, class_teste = calc.calcular_imc(75, 1.75)
calorias_teste = calc.estimar_gasto_calorico(75, "hipertrofia")

print(f"✅ Calculadora criada!")
print(f"🧮 Teste - IMC: {imc_teste} ({class_teste})")
print(f"🔥 Teste - Calorias: {calorias_teste} kcal")

✅ Calculadora criada!
🧮 Teste - IMC: 24.49 (Peso normal)
🔥 Teste - Calorias: 412 kcal


In [ ]:
# 🔍 FERRAMENTA DE BUSCA WEB

class WebSearchTool:
    """Ferramenta para busca web com fallback robusto"""
    
    def __init__(self):
        self.busca_disponivel = BUSCA_WEB_DISPONIVEL
        if self.busca_disponivel:
            try:
                self.ddgs = DDGS()
                print("✅ DuckDuckGo Search inicializado")
            except Exception as e:
                print(f"⚠️ Erro ao inicializar busca: {e}")
                self.busca_disponivel = False
        else:
            self.ddgs = None
    
    def buscar_informacoes_treino(self, objetivo: str, periodicidade: int) -> List[Dict]:
        """Busca informações sobre treinos na web com fallback"""
        
        if not self.busca_disponivel or not self.ddgs:
            # Fallback: retornar informações baseadas em conhecimento interno
            return self._gerar_informacoes_fallback(objetivo, periodicidade)
        
        queries = [
            f"treino {objetivo} {periodicidade} dias semana academia",
            f"exercicios {objetivo} musculacao"
        ]
        
        resultados = []
        for query in queries:
            try:
                search_results = list(self.ddgs.text(
                    keywords=query,
                    region='br',
                    max_results=2
                ))
                
                for result in search_results:
                    resultados.append({
                        "titulo": result.get("title", ""),
                        "resumo": result.get("body", "")[:200] + "...",
                        "query": query
                    })
            except Exception as e:
                print(f"⚠️ Erro na busca '{query}': {e}")
                continue
        
        # Se não conseguiu buscar nada, usar fallback
        if not resultados:
            resultados = self._gerar_informacoes_fallback(objetivo, periodicidade)
                
        return resultados
    
    def _gerar_informacoes_fallback(self, objetivo: str, periodicidade: int) -> List[Dict]:
        """Gera informações baseadas em conhecimento interno"""
        
        dicas_objetivo = {
            "hipertrofia": "Foque em exercícios compostos com 8-12 repetições e sobrecarga progressiva.",
            "emagrecimento": "Combine exercícios de força com alta intensidade e menor descanso.",
            "forca": "Priorize exercícios básicos com 1-6 repetições e cargas altas.",
            "condicionamento": "Use circuitos com exercícios variados e descanso reduzido."
        }
        
        dicas_periodicidade = {
            2: "Full Body 2x permite recuperação adequada para iniciantes.",
            3: "ABC permite trabalhar todos os grupos musculares com boa frequência.",
            4: "ABCD é ideal para intermediários com boa divisão muscular.",
            5: "ABCDE permite foco específico em cada grupo muscular.",
            6: "Push/Pull/Legs 2x maximiza volume e frequência de treino."
        }
        
        return [
            {
                "titulo": f"Treino de {objetivo.title()}",
                "resumo": dicas_objetivo.get(objetivo, "Treino personalizado baseado no seu objetivo."),
                "query": f"treino {objetivo}"
            },
            {
                "titulo": f"Organização {periodicidade}x por semana",
                "resumo": dicas_periodicidade.get(periodicidade, "Organização adequada para sua frequência de treino."),
                "query": f"periodicidade {periodicidade}"
            }
        ]

# Inicializar busca web
web_search = WebSearchTool()
print("✅ Ferramenta de busca criada!")
print(f"🌐 Busca online: {'✅ Ativa' if web_search.busca_disponivel else '❌ Usando fallback'}")

# Teste rápido da busca
teste_busca = web_search.buscar_informacoes_treino("hipertrofia", 4)
print(f"🧪 Teste de busca: {len(teste_busca)} resultados obtidos")

✅ Ferramenta de busca criada!
🔍 Teste de busca será executado nos nós do grafo


/tmp/ipykernel_341801/3773481828.py:7: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  self.ddgs = DDGS()


In [29]:
# 🧠 BASE DE CONHECIMENTO FITNESS (RAG VERDADEIRO)

import os
from pathlib import Path
import numpy as np
from typing import Dict, Any, List, Optional, Tuple
import hashlib
import json

# Verificar se SentenceTransformers está disponível para embeddings
try:
    from sentence_transformers import SentenceTransformer
    EMBEDDINGS_DISPONIVEL = True
except ImportError:
    EMBEDDINGS_DISPONIVEL = False
    print("⚠️ sentence_transformers não disponível - usando busca simples")

def carregar_base_conhecimento():
    """Carrega a base de conhecimento do arquivo externo"""
    arquivo_base = Path("base_conhecimento_fitness.txt")
    
    if not arquivo_base.exists():
        print("⚠️ Arquivo base_conhecimento_fitness.txt não encontrado!")
        return criar_base_basica()
    
    try:
        with open(arquivo_base, 'r', encoding='utf-8') as f:
            conteudo = f.read()
        print(f"✅ Base de conhecimento carregada ({arquivo_base.stat().st_size} bytes)")
        return conteudo
    except Exception as e:
        print(f"❌ Erro ao carregar arquivo: {e}")
        return criar_base_basica()

def criar_base_basica():
    """Base de conhecimento básica como fallback"""
    return """
    ### EXERCÍCIOS BÁSICOS
    PEITO: Supino reto, Flexão de braço
    COSTAS: Puxada frontal, Remada
    PERNAS: Agachamento, Leg press
    """

class FitnessRAG:
    """Sistema RAG completo para conhecimento fitness"""
    
    def __init__(self):
        self.conhecimento_bruto = carregar_base_conhecimento()
        self.chunks = []
        self.embeddings = []
        self.metadata = []
        
        # Inicializar modelo de embeddings se disponível
        if EMBEDDINGS_DISPONIVEL:
            try:
                print("🔄 Carregando modelo de embeddings...")
                self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
                self.usar_embeddings = True
                print("✅ Modelo de embeddings carregado!")
            except Exception as e:
                print(f"⚠️ Erro ao carregar embeddings: {e}")
                self.usar_embeddings = False
        else:
            self.usar_embeddings = False
        
        # Processar documentos
        self._processar_documentos()
        
        # Extrair dados estruturados (fallback)
        self.exercicios_por_grupo = self._extrair_exercicios()
        self.principios_por_objetivo = self._extrair_principios()
    
    def _processar_documentos(self):
        """Processa documentos: chunking + embeddings"""
        print("📄 Processando documentos...")
        
        # 1. CHUNKING: Dividir em pedaços menores
        self.chunks = self._fazer_chunking(self.conhecimento_bruto)
        print(f"✂️ Documento dividido em {len(self.chunks)} chunks")
        
        # 2. EMBEDDINGS: Converter chunks em vetores
        if self.usar_embeddings:
            self._gerar_embeddings()
        
        print("✅ Processamento concluído!")
    
    def _fazer_chunking(self, texto: str, chunk_size: int = 300) -> List[Dict]:
        """Divide texto em chunks menores com metadata"""
        chunks = []
        linhas = texto.split('\n')
        
        chunk_atual = ""
        secao_atual = "Geral"
        
        for linha in linhas:
            linha = linha.strip()
            
            # Detectar nova seção
            if linha.startswith('### '):
                # Salvar chunk anterior se não vazio
                if chunk_atual.strip():
                    chunks.append({
                        'texto': chunk_atual.strip(),
                        'secao': secao_atual,
                        'tokens': len(chunk_atual.split()),
                        'id': len(chunks)
                    })
                
                # Iniciar nova seção
                secao_atual = linha.replace('### ', '').strip()
                chunk_atual = linha + "\\n"
                
            elif linha.startswith('## '):
                # Subtítulo
                if chunk_atual.strip():
                    chunk_atual += linha + "\\n"
            else:
                # Conteúdo regular
                chunk_atual += linha + "\\n"
                
                # Se chunk ficou muito grande, dividir
                if len(chunk_atual.split()) > chunk_size:
                    chunks.append({
                        'texto': chunk_atual.strip(),
                        'secao': secao_atual,
                        'tokens': len(chunk_atual.split()),
                        'id': len(chunks)
                    })
                    chunk_atual = ""
        
        # Adicionar último chunk
        if chunk_atual.strip():
            chunks.append({
                'texto': chunk_atual.strip(),
                'secao': secao_atual,
                'tokens': len(chunk_atual.split()),
                'id': len(chunks)
            })
        
        return chunks
    
    def _gerar_embeddings(self):
        """Gera embeddings vetoriais para os chunks"""
        try:
            print("🧮 Gerando embeddings vetoriais...")
            
            textos = [chunk['texto'] for chunk in self.chunks]
            self.embeddings = self.embedding_model.encode(textos)
            
            # Adicionar embeddings aos metadados
            for i, chunk in enumerate(self.chunks):
                chunk['embedding'] = self.embeddings[i]
            
            print(f"✅ {len(self.embeddings)} embeddings gerados")
            
        except Exception as e:
            print(f"❌ Erro ao gerar embeddings: {e}")
            self.usar_embeddings = False
    
    def buscar_contexto_relevante(self, query: str, top_k: int = 3) -> List[Dict]:
        """Busca contexto relevante usando similaridade vetorial"""
        
        if not self.usar_embeddings or not self.chunks:
            # Fallback: busca por palavras-chave
            return self._busca_palavras_chave(query, top_k)
        
        try:
            # Gerar embedding da query
            query_embedding = self.embedding_model.encode([query])
            
            # Calcular similaridades
            similaridades = []
            for i, chunk in enumerate(self.chunks):
                if 'embedding' in chunk:
                    # Similaridade coseno
                    sim = np.dot(query_embedding[0], chunk['embedding']) / (
                        np.linalg.norm(query_embedding[0]) * np.linalg.norm(chunk['embedding'])
                    )
                    similaridades.append((i, sim, chunk))
            
            # Ordenar por similaridade
            similaridades.sort(key=lambda x: x[1], reverse=True)
            
            # Retornar top-k
            resultados = []
            for i, sim, chunk in similaridades[:top_k]:
                resultados.append({
                    'texto': chunk['texto'],
                    'secao': chunk['secao'],
                    'similaridade': float(sim),
                    'id': chunk['id']
                })
            
            return resultados
            
        except Exception as e:
            print(f"❌ Erro na busca vetorial: {e}")
            return self._busca_palavras_chave(query, top_k)
    
    def _busca_palavras_chave(self, query: str, top_k: int = 3) -> List[Dict]:
        """Busca por palavras-chave (fallback)"""
        query_lower = query.lower()
        query_words = set(query_lower.split())
        
        resultados = []
        for chunk in self.chunks:
            texto_lower = chunk['texto'].lower()
            
            # Contar palavras em comum
            chunk_words = set(texto_lower.split())
            palavras_comuns = len(query_words.intersection(chunk_words))
            
            if palavras_comuns > 0:
                score = palavras_comuns / len(query_words)
                resultados.append({
                    'texto': chunk['texto'],
                    'secao': chunk['secao'],
                    'similaridade': score,
                    'id': chunk['id']
                })
        
        # Ordenar por score
        resultados.sort(key=lambda x: x['similaridade'], reverse=True)
        return resultados[:top_k]
    
    def _extrair_exercicios(self) -> Dict[str, List[str]]:
        """Extrai exercícios estruturados (método anterior mantido)"""
        exercicios = {
            "peito": [], "costas": [], "ombros": [], 
            "bracos": [], "pernas": [], "core": []
        }
        
        linhas = self.conhecimento_bruto.split('\\n')
        grupo_atual = None
        
        for linha in linhas:
            linha = linha.strip()
            
            if '### PEITO' in linha:
                grupo_atual = 'peito'
            elif '### COSTAS' in linha:
                grupo_atual = 'costas'
            elif '### OMBROS' in linha:
                grupo_atual = 'ombros'
            elif '### BRAÇOS' in linha:
                grupo_atual = 'bracos'
            elif '### PERNAS' in linha:
                grupo_atual = 'pernas'
            elif '### CORE' in linha or '### ABDÔMEN' in linha:
                grupo_atual = 'core'
            elif linha.startswith('**') and linha.endswith('**') and grupo_atual:
                exercicio = linha.replace('**', '').strip()
                if exercicio and grupo_atual in exercicios:
                    exercicios[grupo_atual].append(exercicio)
        
        # Fallback se não encontrar
        if all(len(lista) == 0 for lista in exercicios.values()):
            exercicios = {
                "peito": ["Supino reto", "Supino inclinado", "Crucifixo", "Flexão de braço"],
                "costas": ["Puxada frontal", "Remada curvada", "Levantamento terra", "Remada unilateral"],
                "ombros": ["Desenvolvimento militar", "Elevação lateral", "Elevação posterior", "Arnold press"],
                "bracos": ["Rosca direta", "Rosca martelo", "Tríceps testa", "Mergulho"],
                "pernas": ["Agachamento", "Leg press", "Stiff", "Panturrilha"],
                "core": ["Prancha", "Abdominal", "Russian twist"]
            }
        
        return exercicios
    
    def _extrair_principios(self) -> Dict[str, Dict]:
        """Extrai princípios estruturados (método anterior mantido)"""
        principios = {
            "hipertrofia": {"series": "3-4", "repeticoes": "8-12", "descanso": "60-90s"},
            "forca": {"series": "3-5", "repeticoes": "1-6", "descanso": "2-5min"},
            "emagrecimento": {"series": "3-4", "repeticoes": "12-15+", "descanso": "30-60s"},
            "condicionamento": {"series": "3-6", "repeticoes": "15-25", "descanso": "15-45s"}
        }
        return principios
    
    def obter_exercicios(self, grupo: str) -> List[str]:
        """Retorna exercícios para um grupo muscular"""
        return self.exercicios_por_grupo.get(grupo.lower(), [])
    
    def obter_principios(self, objetivo: str) -> Dict:
        """Retorna princípios para um objetivo"""
        return self.principios_por_objetivo.get(objetivo.lower(), {})
    
    def gerar_contexto_personalizado(self, objetivo: str, periodicidade: int) -> str:
        """Gera contexto personalizado usando RAG"""
        
        # Queries específicas para busca
        queries = [
            f"exercícios para {objetivo}",
            f"treino {periodicidade} dias semana",
            f"princípios {objetivo}",
            f"séries repetições {objetivo}"
        ]
        
        contexto_completo = []
        
        for query in queries:
            resultados = self.buscar_contexto_relevante(query, top_k=2)
            for resultado in resultados:
                if resultado['similaridade'] > 0.3:  # Threshold mínimo
                    contexto_completo.append({
                        'texto': resultado['texto'],
                        'relevancia': resultado['similaridade'],
                        'fonte': resultado['secao']
                    })
        
        # Remover duplicatas e ordenar por relevância
        contextos_unicos = {}
        for ctx in contexto_completo:
            hash_texto = hashlib.md5(ctx['texto'].encode()).hexdigest()
            if hash_texto not in contextos_unicos or ctx['relevancia'] > contextos_unicos[hash_texto]['relevancia']:
                contextos_unicos[hash_texto] = ctx
        
        # Compilar contexto final
        contexto_final = "\\n\\n".join([
            f"[{ctx['fonte']}] {ctx['texto']}" 
            for ctx in sorted(contextos_unicos.values(), key=lambda x: x['relevancia'], reverse=True)[:5]
        ])
        
        return contexto_final

# Inicializar sistema RAG
print("🚀 Inicializando sistema RAG completo...")
rag_system = FitnessRAG()

print("\\n✅ Sistema RAG inicializado!")
print(f"📊 Estatísticas:")
print(f"   • Chunks processados: {len(rag_system.chunks)}")
print(f"   • Embeddings: {'✅ Ativo' if rag_system.usar_embeddings else '❌ Desabilitado'}")
print(f"   • Exercícios: {sum(len(ex) for ex in rag_system.exercicios_por_grupo.values())}")
print(f"   • Grupos musculares: {len(rag_system.exercicios_por_grupo)}")

# Teste do RAG
print("\\n🧪 Teste de busca RAG:")
contexto_teste = rag_system.gerar_contexto_personalizado("hipertrofia", 4)
print(f"📝 Contexto gerado ({len(contexto_teste)} caracteres)")

# Exibir alguns exercícios
print(f"\\n💪 Exercícios de peito: {rag_system.obter_exercicios('peito')[:3]}...")
print(f"🎯 Princípios hipertrofia: {rag_system.obter_principios('hipertrofia')}")

🚀 Inicializando sistema RAG completo...
✅ Base de conhecimento carregada (8822 bytes)
🔄 Carregando modelo de embeddings...
✅ Modelo de embeddings carregado!
📄 Processando documentos...
✂️ Documento dividido em 22 chunks
🧮 Gerando embeddings vetoriais...
✅ Modelo de embeddings carregado!
📄 Processando documentos...
✂️ Documento dividido em 22 chunks
🧮 Gerando embeddings vetoriais...
✅ 22 embeddings gerados
✅ Processamento concluído!
\n✅ Sistema RAG inicializado!
📊 Estatísticas:
   • Chunks processados: 22
   • Embeddings: ✅ Ativo
   • Exercícios: 23
   • Grupos musculares: 6
\n🧪 Teste de busca RAG:
📝 Contexto gerado (2115 caracteres)
\n💪 Exercícios de peito: ['Supino reto', 'Supino inclinado', 'Crucifixo']...
🎯 Princípios hipertrofia: {'series': '3-4', 'repeticoes': '8-12', 'descanso': '60-90s'}
✅ 22 embeddings gerados
✅ Processamento concluído!
\n✅ Sistema RAG inicializado!
📊 Estatísticas:
   • Chunks processados: 22
   • Embeddings: ✅ Ativo
   • Exercícios: 23
   • Grupos musculares: 

In [15]:
# 🔍 DEMONSTRAÇÃO DO RAG SEMÂNTICO

print("🧪 TESTE DE BUSCA SEMÂNTICA:")
print("=" * 50)

# Teste 1: Busca por conceitos similares
print("\\n1️⃣ Busca: 'como ganhar músculos'")
resultado1 = rag_system.buscar_contexto_relevante("como ganhar músculos", top_k=3)
for i, resultado in enumerate(resultado1):
    print(f"   📄 Resultado {i+1} (similaridade: {resultado['similaridade']:.3f})")
    print(f"   📂 Seção: {resultado['secao']}")
    print(f"   📝 Texto: {resultado['texto'][:100]}...")
    print()

# Teste 2: Busca por exercícios específicos  
print("\\n2️⃣ Busca: 'exercícios para peito'")
resultado2 = rag_system.buscar_contexto_relevante("exercícios para peito", top_k=2)
for i, resultado in enumerate(resultado2):
    print(f"   📄 Resultado {i+1} (similaridade: {resultado['similaridade']:.3f})")
    print(f"   📂 Seção: {resultado['secao']}")
    print(f"   📝 Texto: {resultado['texto'][:150]}...")
    print()

# Teste 3: Contexto personalizado completo
print("\\n3️⃣ Contexto personalizado para 'hipertrofia + 4 dias':")
contexto_personalizado = rag_system.gerar_contexto_personalizado("hipertrofia", 4)
print(f"📊 Tamanho do contexto: {len(contexto_personalizado)} caracteres")
print(f"📝 Prévia do contexto:")
print(contexto_personalizado[:300] + "..." if len(contexto_personalizado) > 300 else contexto_personalizado)

🧪 TESTE DE BUSCA SEMÂNTICA:
\n1️⃣ Busca: 'como ganhar músculos'
   📄 Resultado 1 (similaridade: 0.552)
   📂 Seção: Geral
   📝 Texto: # BASE DE CONHECIMENTO - FITNESS & MUSCULAÇÃO\n\n## EXERCÍCIOS POR GRUPO MUSCULAR\n\n...

   📄 Resultado 2 (similaridade: 0.456)
   📂 Seção: EXECUÇÃO
   📝 Texto: ### EXECUÇÃO\n- Amplitude completa de movimento\n- Controle nas duas fases (concêntrica e excêntrica...

   📄 Resultado 3 (similaridade: 0.455)
   📂 Seção: GASTO CALÓRICO APROXIMADO POR TREINO
   📝 Texto: ### GASTO CALÓRICO APROXIMADO POR TREINO\n**Musculação (por hora):**\n- Pessoa 70kg: ~220-400 calori...

\n2️⃣ Busca: 'exercícios para peito'
   📄 Resultado 1 (similaridade: 0.473)
   📂 Seção: PEITO
   📝 Texto: ### PEITO\n**Supino reto com barra**\n- Músculos: Peitoral maior, deltóide anterior, tríceps\n- Execução: Deite no banco, pegue a barra com pegada pro...

   📄 Resultado 2 (similaridade: 0.420)
   📂 Seção: EXECUÇÃO
   📝 Texto: ### EXECUÇÃO\n- Amplitude completa de movimento\n- Controle na

---

## 🎯 **3. NÓSNOS DO LANGGRAPH**

Agora vamos criar os 5 nós principais do nosso sistema:

In [17]:
# 📥 NÓ 1: COLETA E VALIDAÇÃO DE DADOS

def no_coleta_validacao(state: FitnessCoachState) -> FitnessCoachState:
    """
    Nó responsável por validar os dados de entrada do usuário
    """
    print("🔍 Executando: Coleta e Validação de Dados")
    
    dados = state["dados_usuario"]
    erros = []
    dados_limpos = {}
    
    # Validar idade
    try:
        idade = int(dados.get("idade", 0))
        if 16 <= idade <= 80:
            dados_limpos["idade"] = idade
        else:
            erros.append("Idade deve estar entre 16 e 80 anos")
    except (ValueError, TypeError):
        erros.append("Idade deve ser um número inteiro")
    
    # Validar peso
    try:
        peso = float(dados.get("peso", 0))
        if 40 <= peso <= 200:
            dados_limpos["peso"] = peso
        else:
            erros.append("Peso deve estar entre 40 e 200 kg")
    except (ValueError, TypeError):
        erros.append("Peso deve ser um número")
    
    # Validar altura (assumindo 1.75m se não fornecida)
    altura = dados.get("altura", 1.75)
    if isinstance(altura, (int, float)) and 1.4 <= altura <= 2.2:
        dados_limpos["altura"] = float(altura)
    else:
        dados_limpos["altura"] = 1.75  # Valor padrão
    
    # Validar periodicidade
    try:
        periodicidade = int(dados.get("periodicidade", 0))
        if 2 <= periodicidade <= 6:
            dados_limpos["periodicidade"] = periodicidade
        else:
            erros.append("Periodicidade deve estar entre 2 e 6 dias por semana")
    except (ValueError, TypeError):
        erros.append("Periodicidade deve ser um número inteiro")
    
    # Validar objetivo
    objetivos_validos = ["hipertrofia", "emagrecimento", "forca", "condicionamento"]
    objetivo = dados.get("objetivo", "").lower()
    if objetivo in objetivos_validos:
        dados_limpos["objetivo"] = objetivo
    else:
        erros.append(f"Objetivo deve ser: {', '.join(objetivos_validos)}")
    
    # Atualizar estado
    validacao_ok = len(erros) == 0
    
    state.update({
        "validacao_ok": validacao_ok,
        "erros_validacao": erros,
        "dados_limpos": dados_limpos,
        "proximo_no": "calculos" if validacao_ok else "erro",
        "mensagens": state.get("mensagens", []) + [f"✅ Validação: {'OK' if validacao_ok else 'ERRO'}"]
    })
    
    return state

# Teste do nó
print("✅ Nó de Coleta e Validação criado!")

✅ Nó de Coleta e Validação criado!


In [18]:
# 🧮 NÓ 2: CÁLCULOS MATEMÁTICOS

def no_calculos_matematicos(state: FitnessCoachState) -> FitnessCoachState:
    """
    Nó responsável por realizar cálculos de IMC, calorias e distribuição
    """
    print("🔢 Executando: Cálculos Matemáticos")
    
    dados = state["dados_limpos"]
    
    # Calcular IMC
    peso = dados["peso"]
    altura = dados["altura"]
    imc, classificacao_imc = calc.calcular_imc(peso, altura)
    
    # Estimar gasto calórico
    objetivo = dados["objetivo"]
    gasto_calorico = calc.estimar_gasto_calorico(peso, objetivo)
    
    # Calcular distribuição de treinos
    periodicidade = dados["periodicidade"]
    distribuicao_treinos = calc.calcular_distribuicao_treinos(periodicidade)
    
    # Atualizar estado
    state.update({
        "imc": imc,
        "classificacao_imc": classificacao_imc,
        "gasto_calorico": gasto_calorico,
        "distribuicao_treinos": distribuicao_treinos,
        "proximo_no": "busca_web",
        "mensagens": state.get("mensagens", []) + [f"🧮 Cálculos: IMC {imc}, {gasto_calorico} kcal"]
    })
    
    return state

print("✅ Nó de Cálculos Matemáticos criado!")

✅ Nó de Cálculos Matemáticos criado!


In [19]:
# 🌐 NÓ 3: BUSCA WEB

def no_busca_web(state: FitnessCoachState) -> FitnessCoachState:
    """
    Nó responsável por buscar informações na web sobre treinos
    """
    print("🔍 Executando: Busca Web")
    
    dados = state["dados_limpos"]
    objetivo = dados["objetivo"]
    periodicidade = dados["periodicidade"]
    
    informacoes_web = []
    
    if web_search:
        try:
            resultados = web_search.buscar_informacoes_treino(objetivo, periodicidade)
            informacoes_web = resultados[:3]  # Máximo 3 resultados
        except Exception as e:
            print(f"⚠️ Erro na busca web: {e}")
            informacoes_web = [{
                "titulo": "Busca offline",
                "resumo": "Usando conhecimento interno devido à indisponibilidade da busca web",
                "query": "offline"
            }]
    else:
        informacoes_web = [{
            "titulo": "Modo offline",
            "resumo": "Utilizando base de conhecimento interna para gerar treinos personalizados",
            "query": "offline"
        }]
    
    # Atualizar estado
    state.update({
        "informacoes_web": informacoes_web,
        "proximo_no": "rag",
        "mensagens": state.get("mensagens", []) + [f"🌐 Busca: {len(informacoes_web)} resultados"]
    })
    
    return state

print("✅ Nó de Busca Web criado!")

✅ Nó de Busca Web criado!


In [20]:
# 🧠 NÓ 4: SISTEMA RAG AVANÇADO

def no_rag(state: FitnessCoachState) -> FitnessCoachState:
    """
    Nó responsável por recuperar conhecimento usando RAG semântico
    """
    print("🧠 Executando: Sistema RAG Avançado")
    
    dados = state["dados_limpos"]
    objetivo = dados["objetivo"]
    periodicidade = dados["periodicidade"]
    
    # 🔍 Gerar contexto personalizado usando RAG semântico
    contexto_rag = rag_system.gerar_contexto_personalizado(objetivo, periodicidade)
    
    # 📊 Buscar informações específicas por objetivo
    queries_especificas = [
        f"princípios treino {objetivo}",
        f"séries repetições {objetivo}",
        f"exercícios {objetivo}",
        f"descanso intervalo {objetivo}"
    ]
    
    conhecimento_detalhado = []
    for query in queries_especificas:
        resultados = rag_system.buscar_contexto_relevante(query, top_k=1)
        if resultados and resultados[0]['similaridade'] > 0.3:
            conhecimento_detalhado.append(resultados[0]['texto'])
    
    # 💪 Obter exercícios usando sistema estruturado (fallback confiável)
    grupos_treino = {
        2: ["peito", "costas"],  # Full body alternado
        3: ["peito", "costas", "pernas"],  # ABC
        4: ["peito", "costas", "ombros", "pernas"],  # ABCD
        5: ["peito", "costas", "ombros", "bracos", "pernas"],  # ABCDE
        6: ["peito", "costas", "ombros", "bracos", "pernas", "pernas"]  # Push/Pull/Legs 2x
    }
    
    grupos_selecionados = grupos_treino.get(periodicidade, grupos_treino[4])
    
    # 🏋️ Obter exercícios para cada grupo
    exercicios_sugeridos = {}
    for grupo in grupos_selecionados:
        exercicios = rag_system.obter_exercicios(grupo)
        
        # Se não encontrou exercícios, usar busca RAG
        if not exercicios:
            resultados_exercicios = rag_system.buscar_contexto_relevante(f"exercícios {grupo}", top_k=1)
            if resultados_exercicios:
                # Extrair exercícios do contexto encontrado
                texto = resultados_exercicios[0]['texto']
                linhas = texto.split('\\n')
                exercicios_encontrados = []
                for linha in linhas:
                    if linha.startswith('**') and linha.endswith('**'):
                        exercicio = linha.replace('**', '').strip()
                        if exercicio:
                            exercicios_encontrados.append(exercicio)
                exercicios = exercicios_encontrados[:6]  # Máximo 6
        
        exercicios_sugeridos[grupo] = exercicios if exercicios else [f"Exercício para {grupo}"]
    
    # 🎯 Obter princípios de treino
    principios_treino = rag_system.obter_principios(objetivo)
    
    # 📈 Adicionar insights do RAG aos princípios
    if contexto_rag and len(contexto_rag) > 100:
        # Extrair insights específicos do contexto RAG
        if "séries" in contexto_rag.lower():
            # Buscar padrões de séries no contexto
            import re
            series_match = re.search(r'(\\d+-\\d+)\\s*séries', contexto_rag.lower())
            if series_match and 'series' not in principios_treino:
                principios_treino['series'] = series_match.group(1)
        
        if "repetições" in contexto_rag.lower():
            # Buscar padrões de repetições
            reps_match = re.search(r'(\\d+-\\d+[+]?)\\s*repetições', contexto_rag.lower())
            if reps_match and 'repeticoes' not in principios_treino:
                principios_treino['repeticoes'] = reps_match.group(1)
    
    # 📝 Criar conhecimento estruturado
    conhecimento_rag_estruturado = [
        f"Treino focado em {objetivo}",
        f"Periodicidade: {periodicidade} dias por semana", 
        f"Distribuição: {state['distribuicao_treinos']['tipo']}",
        f"Contexto RAG: {len(contexto_rag)} caracteres de conhecimento específico",
        f"Exercícios encontrados: {sum(len(ex) for ex in exercicios_sugeridos.values())} exercícios"
    ]
    
    # Se temos conhecimento detalhado, adicionar resumo
    if conhecimento_detalhado:
        conhecimento_rag_estruturado.append(f"Insights específicos: {len(conhecimento_detalhado)} seções relevantes")
    
    # 🔄 Atualizar estado
    state.update({
        "conhecimento_rag": conhecimento_rag_estruturado,
        "exercicios_sugeridos": exercicios_sugeridos,
        "principios_treino": principios_treino,
        "contexto_rag_completo": contexto_rag,  # ← Novo: contexto RAG completo
        "insights_rag": conhecimento_detalhado,  # ← Novo: insights específicos
        "proximo_no": "geracao",
        "mensagens": state.get("mensagens", []) + [
            f"🧠 RAG: {len(exercicios_sugeridos)} grupos, contexto de {len(contexto_rag)} chars"
        ]
    })
    
    return state

print("✅ Nó RAG Avançado criado!")
print("🔧 Funcionalidades:")
print("   • Busca semântica por similaridade")
print("   • Contexto personalizado por objetivo") 
print("   • Extração automática de princípios")
print("   • Fallback robusto para exercícios")
print("   • Insights específicos do RAG")

✅ Nó RAG Avançado criado!
🔧 Funcionalidades:
   • Busca semântica por similaridade
   • Contexto personalizado por objetivo
   • Extração automática de princípios
   • Fallback robusto para exercícios
   • Insights específicos do RAG


In [21]:
# 📝 NÓ 5: GERAÇÃO DE PLANILHA

def no_geracao_planilha(state: FitnessCoachState) -> FitnessCoachState:
    """
    Nó responsável por gerar a planilha completa de 12 treinos
    """
    print("📝 Executando: Geração de Planilha")
    
    dados = state["dados_limpos"]
    exercicios = state["exercicios_sugeridos"]
    principios = state["principios_treino"]
    periodicidade = dados["periodicidade"]
    
    planilha_treinos = []
    
    # Gerar 12 treinos
    for i in range(1, 13):
        # Determinar qual grupo muscular trabalhar (rotação)
        grupos_disponiveis = list(exercicios.keys())
        grupo_atual = grupos_disponiveis[(i - 1) % len(grupos_disponiveis)]
        
        # Selecionar exercícios para este treino
        exercicios_grupo = exercicios[grupo_atual]
        exercicios_treino = exercicios_grupo[:4] if len(exercicios_grupo) >= 4 else exercicios_grupo
        
        # Adicionar exercícios complementares se necessário
        if len(exercicios_treino) < 4:
            # Pegar exercícios de outros grupos para completar
            for outro_grupo in grupos_disponiveis:
                if outro_grupo != grupo_atual:
                    exercicios_treino.extend(exercicios[outro_grupo][:2])
                    if len(exercicios_treino) >= 4:
                        exercicios_treino = exercicios_treino[:4]
                        break
        
        # Criar estrutura do treino
        treino = {
            "numero": i,
            "nome": f"Treino {chr(64 + ((i-1) % periodicidade) + 1)} - {grupo_atual.title()}",
            "grupo_principal": grupo_atual.title(),
            "exercicios": []
        }
        
        # Adicionar exercícios com séries e repetições
        for j, exercicio in enumerate(exercicios_treino[:6]):  # Máximo 6 exercícios
            exercicio_info = {
                "ordem": j + 1,
                "nome": exercicio,
                "series": principios.get("series", "3"),
                "repeticoes": principios.get("repeticoes", "8-12"),
                "descanso": principios.get("descanso", "60s"),
                "observacao": f"Exercício para {grupo_atual}"
            }
            treino["exercicios"].append(exercicio_info)
        
        planilha_treinos.append(treino)
    
    # Criar resumo final
    resumo_final = f"""
    🏋️‍♂️ PLANILHA PERSONALIZADA - AI FITNESS COACH
    
    👤 PERFIL DO USUÁRIO:
    • Idade: {dados['idade']} anos
    • Peso: {dados['peso']} kg
    • Altura: {dados['altura']} m
    • IMC: {state['imc']} ({state['classificacao_imc']})
    • Objetivo: {dados['objetivo'].title()}
    • Frequência: {periodicidade} dias/semana
    
    🎯 PARÂMETROS DO TREINO:
    • Séries: {principios.get('series', '3')}
    • Repetições: {principios.get('repeticoes', '8-12')}
    • Descanso: {principios.get('descanso', '60s')}
    • Gasto Calórico/Treino: ~{state['gasto_calorico']} kcal
    
    📊 ESTRUTURA:
    • Total de treinos: 12
    • Tipo de divisão: {state['distribuicao_treinos']['tipo']}
    • Ciclos completos: {state['distribuicao_treinos']['ciclos']}
    """
    
    # Atualizar estado
    state.update({
        "planilha_treinos": planilha_treinos,
        "resumo_final": resumo_final,
        "proximo_no": "fim",
        "mensagens": state.get("mensagens", []) + [f"📝 Geração: 12 treinos criados!"]
    })
    
    return state

print("✅ Nó de Geração de Planilha criado!")

✅ Nó de Geração de Planilha criado!


---

## 🔗 **4. CONSTRUÇÃO DO GRAFO LANGGRAPH**

Agora vamos conectar todos os nós em um fluxo de trabalho orquestrado:

In [22]:
# 🔗 CONSTRUÇÃO DO GRAFO LANGGRAPH

def criar_fitness_coach_graph():
    """
    Cria e configura o grafo LangGraph para o AI Fitness Coach
    """
    
    # Criar o grafo
    workflow = StateGraph(FitnessCoachState)
    
    # Adicionar todos os nós
    workflow.add_node("validacao", no_coleta_validacao)
    workflow.add_node("calculos", no_calculos_matematicos)
    workflow.add_node("busca_web", no_busca_web)
    workflow.add_node("rag", no_rag)
    workflow.add_node("geracao", no_geracao_planilha)
    
    # Definir ponto de entrada
    workflow.set_entry_point("validacao")
    
    # Definir as conexões entre nós
    workflow.add_edge("validacao", "calculos")
    workflow.add_edge("calculos", "busca_web")
    workflow.add_edge("busca_web", "rag")
    workflow.add_edge("rag", "geracao")
    workflow.add_edge("geracao", END)
    
    # Compilar o grafo
    app = workflow.compile()
    
    return app

# Criar o grafo
fitness_coach_app = criar_fitness_coach_graph()

print("✅ Grafo LangGraph criado com sucesso!")
print("🔗 Fluxo: Validação → Cálculos → Busca Web → RAG → Geração → FIM")

✅ Grafo LangGraph criado com sucesso!
🔗 Fluxo: Validação → Cálculos → Busca Web → RAG → Geração → FIM


In [25]:
# 🧪 TESTE DO SISTEMA COMPLETO COM RAG VERDADEIRO

print("🚀 TESTANDO AI FITNESS COACH COM RAG SEMÂNTICO")
print("=" * 60)

# Dados de teste focados em hipertrofia
dados_rag_teste = {
    "idade": 26,
    "peso": 78.0,
    "altura": 1.78,
    "periodicidade": 4,  # ABCD
    "objetivo": "hipertrofia"
}

print("👤 Perfil de teste:")
for campo, valor in dados_rag_teste.items():
    print(f"   {campo.title()}: {valor}")

print("\\n🔄 Executando pipeline com RAG avançado...")
resultado_rag = executar_fitness_coach(dados_rag_teste)

if resultado_rag:
    print("\\n🎯 RESULTADOS DO RAG:")
    print(f"   📊 IMC: {resultado_rag['imc']} ({resultado_rag['classificacao_imc']})")
    print(f"   🔥 Gasto calórico: {resultado_rag['gasto_calorico']} kcal/treino")
    print(f"   📝 Contexto RAG: {len(resultado_rag.get('contexto_rag_completo', ''))} caracteres")
    print(f"   🧠 Insights RAG: {len(resultado_rag.get('insights_rag', []))} seções específicas")
    
    print("\\n💪 EXERCÍCIOS EXTRAÍDOS VIA RAG:")
    exercicios = resultado_rag.get('exercicios_sugeridos', {})
    for grupo, lista_ex in exercicios.items():
        print(f"   {grupo.title()}: {', '.join(lista_ex[:3])}{'...' if len(lista_ex) > 3 else ''}")
    
    print("\\n📊 PRINCÍPIOS DE TREINO:")
    principios = resultado_rag.get('principios_treino', {})
    for param, valor in principios.items():
        print(f"   {param.title()}: {valor}")

else:
    print("❌ Erro no teste do RAG")
    
print("\\n" + "="*60)

🚀 TESTANDO AI FITNESS COACH COM RAG SEMÂNTICO
👤 Perfil de teste:
   Idade: 26
   Peso: 78.0
   Altura: 1.78
   Periodicidade: 4
   Objetivo: hipertrofia
\n🔄 Executando pipeline com RAG avançado...
🏋️‍♂️ INICIANDO AI FITNESS COACH
🔄 Executando pipeline com RAG semântico...
🔍 Executando: Coleta e Validação de Dados
🔢 Executando: Cálculos Matemáticos
🔍 Executando: Busca Web
🧠 Executando: Sistema RAG Avançado
📝 Executando: Geração de Planilha
✅ Validação: OK\n🧮 Cálculos: IMC 24.62, 429 kcal\n🌐 Busca: 3 resultados\n🧠 RAG: 4 grupos, contexto de 2115 chars\n📝 Geração: 12 treinos criados!
\n✅ Pipeline executado com sucesso!
\n🎯 RESULTADOS DO RAG:
   📊 IMC: 24.62 (Peso normal)
   🔥 Gasto calórico: 429 kcal/treino
   📝 Contexto RAG: 0 caracteres
   🧠 Insights RAG: 0 seções específicas
\n💪 EXERCÍCIOS EXTRAÍDOS VIA RAG:
   Peito: Supino reto, Supino inclinado, Crucifixo...
   Costas: Puxada frontal, Remada curvada, Levantamento terra...
   Ombros: Desenvolvimento militar, Elevação lateral, Elevaçã

---

## 🎨 **5. INTERFACE INTERATIVA**

Agora vamos criar uma interface amigável para coletar os dados do usuário:

In [ ]:
# 🎨 INTERFACE DE COLETA DE DADOS

def coletar_dados_usuario():
    """
    Interface interativa para coletar dados do usuário
    """
    print("🏋️‍♂️ BEM-VINDO AO AI FITNESS COACH! 🏋️‍♂️")
    print("=" * 50)
    print("Vamos criar sua planilha personalizada de treinos!")
    print()
    
    dados = {}
    
    # Coletar idade
    while True:
        try:
            idade = input("📅 Qual sua idade? (16-80 anos): ")
            idade = int(idade)
            if 16 <= idade <= 80:
                dados["idade"] = idade
                break
            else:
                print("❌ Idade deve estar entre 16 e 80 anos")
        except ValueError:
            print("❌ Por favor, digite apenas números")
    
    # Coletar peso
    while True:
        try:
            peso = input("⚖️ Qual seu peso? (kg): ")
            peso = float(peso.replace(",", "."))
            if 40 <= peso <= 200:
                dados["peso"] = peso
                break
            else:
                print("❌ Peso deve estar entre 40 e 200 kg")
        except ValueError:
            print("❌ Por favor, digite um número válido")
    
    # Coletar altura (opcional)
    while True:
        try:
            altura_input = input("📏 Qual sua altura? (m) [Enter para 1.75]: ")
            if altura_input.strip() == "":
                dados["altura"] = 1.75
                break
            else:
                altura = float(altura_input.replace(",", "."))
                if 1.4 <= altura <= 2.2:
                    dados["altura"] = altura
                    break
                else:
                    print("❌ Altura deve estar entre 1.4 e 2.2 metros")
        except ValueError:
            print("❌ Por favor, digite um número válido")
    
    # Coletar periodicidade
    print("\\n📅 Quantos dias por semana você pode treinar?")
    print("2 - Iniciante (Full Body)")
    print("3 - Básico (ABC)")
    print("4 - Intermediário (ABCD)")
    print("5 - Avançado (ABCDE)")
    print("6 - Muito avançado (Push/Pull/Legs 2x)")
    
    while True:
        try:
            periodicidade = input("Escolha (2-6): ")
            periodicidade = int(periodicidade)
            if 2 <= periodicidade <= 6:
                dados["periodicidade"] = periodicidade
                break
            else:
                print("❌ Escolha entre 2 e 6 dias")
        except ValueError:
            print("❌ Por favor, digite apenas números")
    
    # Coletar objetivo
    print("\\n🎯 Qual seu objetivo principal?")
    print("1 - Hipertrofia (ganhar músculos)")
    print("2 - Emagrecimento (perder gordura)")
    print("3 - Força (ficar mais forte)")
    print("4 - Condicionamento (resistência)")
    
    objetivos_map = {
        "1": "hipertrofia",
        "2": "emagrecimento", 
        "3": "forca",
        "4": "condicionamento"
    }
    
    while True:
        escolha = input("Escolha (1-4): ")
        if escolha in objetivos_map:
            dados["objetivo"] = objetivos_map[escolha]
            break
        else:
            print("❌ Escolha entre 1 e 4")
    
    print("\\n✅ Dados coletados com sucesso!")
    return dados

# Função auxiliar para execução não-interativa (para demonstração)
def dados_exemplo():
    """Dados de exemplo para demonstração"""
    return {
        "idade": 28,
        "peso": 75.0,
        "altura": 1.75,
        "periodicidade": 4,
        "objetivo": "hipertrofia"
    }

print("✅ Interface de coleta criada!")
print("💡 Use coletar_dados_usuario() para interface interativa")
print("💡 Use dados_exemplo() para teste rápido")

---

## 🚀 **6. EXECUÇÃO DO SISTEMA**

Agora vamos executar o AI Fitness Coach completo!

In [24]:
# 🚀 EXECUÇÃO PRINCIPAL DO AI FITNESS COACH

def executar_fitness_coach(dados_usuario):
    """
    Executa o pipeline completo do AI Fitness Coach com RAG avançado
    """
    print("🏋️‍♂️ INICIANDO AI FITNESS COACH")
    print("=" * 50)
    
    # Estado inicial com novos campos RAG
    estado_inicial = {
        "dados_usuario": dados_usuario,
        "validacao_ok": False,
        "erros_validacao": [],
        "dados_limpos": {},
        "imc": 0.0,
        "classificacao_imc": "",
        "gasto_calorico": 0,
        "distribuicao_treinos": {},
        "informacoes_web": [],
        "conhecimento_rag": [],
        "exercicios_sugeridos": {},
        "principios_treino": {},
        "contexto_rag_completo": "",  # ← Novo campo RAG
        "insights_rag": [],           # ← Novo campo RAG
        "planilha_treinos": [],
        "resumo_final": "",
        "proximo_no": "validacao",
        "mensagens": []
    }
    
    # Executar o grafo
    print("🔄 Executando pipeline com RAG semântico...")
    try:
        resultado = fitness_coach_app.invoke(estado_inicial)
        
        print("\\n".join(resultado["mensagens"]))
        print("\\n✅ Pipeline executado com sucesso!")
        
        # Estatísticas do RAG
        if "contexto_rag_completo" in resultado:
            contexto_size = len(resultado["contexto_rag_completo"])
            insights_count = len(resultado.get("insights_rag", []))
            print(f"🧠 RAG Stats: {contexto_size} chars contexto, {insights_count} insights")
        
        return resultado
        
    except Exception as e:
        print(f"❌ Erro na execução: {e}")
        import traceback
        print(f"🔍 Detalhes: {traceback.format_exc()}")
        return None

def exibir_planilha_avancada(resultado):
    """
    Exibe a planilha de treinos com informações RAG
    """
    if not resultado or not resultado.get("planilha_treinos"):
        print("❌ Nenhuma planilha para exibir")
        return
    
    # Exibir resumo
    print(resultado["resumo_final"])
    
    # Exibir informações RAG se disponível
    if resultado.get("contexto_rag_completo"):
        print(f"\\n🧠 CONTEXTO RAG UTILIZADO:")
        print("-" * 40)
        contexto = resultado["contexto_rag_completo"]
        # Mostrar primeiras linhas do contexto
        linhas_contexto = contexto.split('\\n')[:5]
        for linha in linhas_contexto:
            if linha.strip():
                print(f"📝 {linha.strip()}")
        if len(linhas_contexto) < len(contexto.split('\\n')):
            print(f"📊 ... e mais {len(contexto.split('\\n')) - len(linhas_contexto)} linhas")
    
    print("\\n" + "="*80)
    print("📋 PLANILHA COMPLETA - 12 TREINOS")
    print("="*80)
    
    # Exibir cada treino
    for treino in resultado["planilha_treinos"]:
        print(f"\\n🏋️‍♂️ {treino['nome']}")
        print("-" * 40)
        
        for exercicio in treino["exercicios"]:
            print(f"{exercicio['ordem']}. {exercicio['nome']}")
            print(f"   📊 {exercicio['series']} séries x {exercicio['repeticoes']} reps")
            print(f"   ⏱️ Descanso: {exercicio['descanso']}")
        
        print()

# Exemplo de uso
print("✅ Função de execução RAG criada!")
print("💡 Para executar:")
print("   1. dados = coletar_dados_usuario()  # ou dados_exemplo()")
print("   2. resultado = executar_fitness_coach(dados)")
print("   3. exibir_planilha_avancada(resultado)")
print("\\n🆕 Novidades:")
print("   • RAG semântico com embeddings")
print("   • Busca por similaridade vetorial")
print("   • Contexto personalizado inteligente") 
print("   • Extração automática de princípios")

✅ Função de execução RAG criada!
💡 Para executar:
   1. dados = coletar_dados_usuario()  # ou dados_exemplo()
   2. resultado = executar_fitness_coach(dados)
   3. exibir_planilha_avancada(resultado)
\n🆕 Novidades:
   • RAG semântico com embeddings
   • Busca por similaridade vetorial
   • Contexto personalizado inteligente
   • Extração automática de princípios


---

## 🧪 **7. DEMONSTRAÇÃO E TESTES**

Vamos testar o sistema com diferentes cenários:

In [ ]:
# 🧪 TESTE 1: CENÁRIO HIPERTROFIA

print("🧪 TESTE 1: Usuário focado em HIPERTROFIA")
print("=" * 50)

# Dados de teste: jovem querendo ganhar músculos
dados_hipertrofia = {
    "idade": 25,
    "peso": 70.0,
    "altura": 1.80,
    "periodicidade": 4,  # 4x por semana
    "objetivo": "hipertrofia"
}

print("👤 Perfil do teste:")
print(f"   Idade: {dados_hipertrofia['idade']} anos")
print(f"   Peso: {dados_hipertrofia['peso']} kg")
print(f"   Altura: {dados_hipertrofia['altura']} m")
print(f"   Treinos: {dados_hipertrofia['periodicidade']} dias/semana")
print(f"   Objetivo: {dados_hipertrofia['objetivo'].title()}")

print("\\n🚀 Executando...")

# Execute esta célula para ver o resultado
resultado_hipertrofia = executar_fitness_coach(dados_hipertrofia)

if resultado_hipertrofia:
    print("\\n📊 RESULTADOS:")
    print(f"   IMC: {resultado_hipertrofia['imc']} ({resultado_hipertrofia['classificacao_imc']})")
    print(f"   Gasto calórico/treino: {resultado_hipertrofia['gasto_calorico']} kcal")
    print(f"   Total de treinos gerados: {len(resultado_hipertrofia['planilha_treinos'])}")
    
    print("\\n🏋️‍♂️ PRIMEIROS 2 TREINOS:")
    for i, treino in enumerate(resultado_hipertrofia['planilha_treinos'][:2]):
        print(f"\\n{treino['nome']}:")
        for ex in treino['exercicios'][:3]:  # Mostrar só 3 exercícios
            print(f"  • {ex['nome']} - {ex['series']} x {ex['repeticoes']}")
else:
    print("❌ Erro na execução do teste")

In [ ]:
# 🧪 TESTE 2: CENÁRIO EMAGRECIMENTO

print("🧪 TESTE 2: Usuário focado em EMAGRECIMENTO")
print("=" * 50)

# Dados de teste: pessoa querendo perder peso
dados_emagrecimento = {
    "idade": 35,
    "peso": 85.0,
    "altura": 1.70,
    "periodicidade": 5,  # 5x por semana
    "objetivo": "emagrecimento"
}

print("👤 Perfil do teste:")
for key, value in dados_emagrecimento.items():
    print(f"   {key.title()}: {value}")

print("\\n🚀 Executando...")
resultado_emagrecimento = executar_fitness_coach(dados_emagrecimento)

if resultado_emagrecimento:
    print("\\n📊 COMPARAÇÃO DOS RESULTADOS:")
    print("HIPERTROFIA vs EMAGRECIMENTO:")
    print(f"Calorias/treino: {resultado_hipertrofia['gasto_calorico']} vs {resultado_emagrecimento['gasto_calorico']}")
    
    # Comparar princípios de treino
    princ_hiper = resultado_hipertrofia['principios_treino']
    princ_emag = resultado_emagrecimento['principios_treino']
    
    print(f"Repetições: {princ_hiper.get('repeticoes', 'N/A')} vs {princ_emag.get('repeticoes', 'N/A')}")
    print(f"Descanso: {princ_hiper.get('descanso', 'N/A')} vs {princ_emag.get('descanso', 'N/A')}")
else:
    print("❌ Erro na execução do teste")

In [ ]:
# 🎯 EXECUÇÃO INTERATIVA - CRIE SUA PLANILHA!

print("🎯 AGORA É SUA VEZ!")
print("=" * 50)
print("Execute a célula abaixo para usar o sistema interativo")
print("ou modifique os dados de exemplo para seu perfil")
print()

# Para uso interativo (descomente para usar):
# dados_usuario = coletar_dados_usuario()

# Para teste rápido com seus dados:
meus_dados = {
    "idade": 28,        # Sua idade
    "peso": 75.0,       # Seu peso
    "altura": 1.75,     # Sua altura
    "periodicidade": 4, # Quantos dias quer treinar (2-6)
    "objetivo": "hipertrofia"  # hipertrofia, emagrecimento, forca, condicionamento
}

print("📝 Usando dados personalizados:")
for campo, valor in meus_dados.items():
    print(f"   {campo.title()}: {valor}")

print("\\n🚀 Gerando sua planilha personalizada...")
meu_resultado = executar_fitness_coach(meus_dados)

print("\\n🎉 SUA PLANILHA ESTÁ PRONTA!")
print("Execute a próxima célula para visualizar completa")

In [ ]:
# 📋 VISUALIZAÇÃO COMPLETA DA PLANILHA

# Exibir planilha completa
if 'meu_resultado' in locals() and meu_resultado:
    exibir_planilha_avancada(meu_resultado)
else:
    print("⚠️ Execute a célula anterior primeiro para gerar a planilha")

print("\\n" + "="*80)
print("🎊 PARABÉNS! Sua planilha personalizada está completa!")
print("="*80)
print("\\n📱 PRÓXIMOS PASSOS:")
print("   1. 📋 Salve esta planilha")
print("   2. 🏋️‍♂️ Comece seus treinos")  
print("   3. 📈 Acompanhe seu progresso")
print("   4. 🔄 Ajuste conforme necessário")

print("\\n💡 DICAS:")
print("   • Sempre aqueça antes de treinar")
print("   • Mantenha boa forma nos exercícios")
print("   • Descanse adequadamente entre treinos")
print("   • Ajuste cargas progressivamente")
print("   • Consulte um profissional se necessário")

---

## 💾 **8. SALVAMENTO DA PLANILHA**

Vamos implementar a funcionalidade de salvar a planilha em arquivo:

In [ ]:
# 💾 FUNCIONALIDADE DE SALVAMENTO

def salvar_planilha(resultado, nome_arquivo="minha_planilha_treino"):
    """
    Salva a planilha de treino em arquivo de texto
    """
    if not resultado or not resultado.get("planilha_treinos"):
        print("❌ Nenhuma planilha para salvar")
        return False
    
    try:
        # Criar conteúdo do arquivo
        conteudo = []
        conteudo.append("=" * 80)
        conteudo.append("🏋️‍♂️ AI FITNESS COACH - PLANILHA PERSONALIZADA")
        conteudo.append("=" * 80)
        conteudo.append("")
        
        # Adicionar resumo
        conteudo.append(resultado["resumo_final"])
        conteudo.append("")
        
        # Adicionar contexto RAG se disponível
        if resultado.get("contexto_rag_completo"):
            conteudo.append("🧠 CONTEXTO RAG UTILIZADO:")
            conteudo.append("-" * 40)
            contexto = resultado["contexto_rag_completo"]
            linhas_contexto = contexto.split('\n')[:10]  # Primeiras 10 linhas
            conteudo.extend([f"📝 {linha}" for linha in linhas_contexto if linha.strip()])
            conteudo.append("")
        
        conteudo.append("=" * 80)
        conteudo.append("📋 PLANILHA COMPLETA - 12 TREINOS")
        conteudo.append("=" * 80)
        conteudo.append("")
        
        # Adicionar cada treino
        for i, treino in enumerate(resultado["planilha_treinos"], 1):
            conteudo.append(f"🏋️‍♂️ {treino['nome']}")
            conteudo.append("-" * 50)
            
            for exercicio in treino["exercicios"]:
                conteudo.append(f"{exercicio['ordem']}. {exercicio['nome']}")
                conteudo.append(f"   📊 {exercicio['series']} séries x {exercicio['repeticoes']} repetições")
                conteudo.append(f"   ⏱️ Descanso: {exercicio['descanso']}")
                conteudo.append("")
            
            conteudo.append("")
        
        # Adicionar rodapé
        conteudo.append("=" * 80)
        conteudo.append("🎯 DICAS IMPORTANTES:")
        conteudo.append("• Sempre aqueça antes de treinar")
        conteudo.append("• Mantenha boa forma nos exercícios")
        conteudo.append("• Descanse adequadamente entre treinos")
        conteudo.append("• Ajuste cargas progressivamente")
        conteudo.append("• Consulte um profissional se necessário")
        conteudo.append("")
        conteudo.append(f"📅 Gerado em: {datetime.now().strftime('%d/%m/%Y às %H:%M:%S')}")
        conteudo.append("🤖 AI Fitness Coach - Powered by LangGraph")
        conteudo.append("=" * 80)
        
        # Salvar arquivo
        nome_completo = f"{nome_arquivo}.txt"
        with open(nome_completo, 'w', encoding='utf-8') as f:
            f.write('\n'.join(conteudo))
        
        print(f"✅ Planilha salva com sucesso!")
        print(f"📁 Arquivo: {nome_completo}")
        print(f"📊 Tamanho: {len('\n'.join(conteudo))} caracteres")
        
        return True
        
    except Exception as e:
        print(f"❌ Erro ao salvar planilha: {e}")
        return False

def gerar_relatorio_completo(resultado):
    """
    Gera relatório detalhado com todas as informações do sistema
    """
    if not resultado:
        print("❌ Nenhum resultado para gerar relatório")
        return
    
    print("📊 RELATÓRIO COMPLETO DO AI FITNESS COACH")
    print("=" * 60)
    
    # Dados do usuário
    dados = resultado.get("dados_limpos", {})
    print(f"👤 USUÁRIO:")
    print(f"   • Idade: {dados.get('idade', 'N/A')} anos")
    print(f"   • Peso: {dados.get('peso', 'N/A')} kg")
    print(f"   • Altura: {dados.get('altura', 'N/A')} m")
    print(f"   • Objetivo: {dados.get('objetivo', 'N/A').title()}")
    print(f"   • Frequência: {dados.get('periodicidade', 'N/A')} dias/semana")
    
    # Métricas calculadas
    print(f"\n📈 MÉTRICAS:")
    print(f"   • IMC: {resultado.get('imc', 'N/A')} ({resultado.get('classificacao_imc', 'N/A')})")
    print(f"   • Gasto calórico/treino: ~{resultado.get('gasto_calorico', 'N/A')} kcal")
    
    # Informações do sistema
    distribuicao = resultado.get('distribuicao_treinos', {})
    print(f"\n🏗️ SISTEMA:")
    print(f"   • Tipo de divisão: {distribuicao.get('tipo', 'N/A')}")
    print(f"   • Ciclos completos: {distribuicao.get('ciclos', 'N/A')}")
    print(f"   • Total de treinos: {len(resultado.get('planilha_treinos', []))}")
    
    # Estatísticas RAG
    exercicios = resultado.get('exercicios_sugeridos', {})
    total_exercicios = sum(len(lista) for lista in exercicios.values())
    print(f"\n🧠 RAG STATS:")
    print(f"   • Grupos musculares: {len(exercicios)}")
    print(f"   • Total de exercícios: {total_exercicios}")
    print(f"   • Contexto RAG: {len(resultado.get('contexto_rag_completo', ''))} chars")
    print(f"   • Insights extraídos: {len(resultado.get('insights_rag', []))}")
    
    # Busca web
    web_info = resultado.get('informacoes_web', [])
    print(f"\n🌐 BUSCA WEB:")
    print(f"   • Resultados obtidos: {len(web_info)}")
    for i, info in enumerate(web_info[:2], 1):
        print(f"   • Resultado {i}: {info.get('titulo', 'N/A')[:50]}...")
    
    print("\n" + "=" * 60)

print("✅ Funcionalidades de salvamento criadas!")
print("💾 Use salvar_planilha(resultado) para salvar em arquivo")
print("📊 Use gerar_relatorio_completo(resultado) para relatório detalhado")

---

## 🚀 **9. DEMONSTRAÇÃO FINAL COMPLETA**

Agora vamos executar o sistema completo do início ao fim:

In [ ]:
# 🚀 DEMONSTRAÇÃO FINAL - SISTEMA COMPLETO

def demonstracao_completa():
    """
    Executa uma demonstração completa do AI Fitness Coach
    """
    print("🎯 DEMONSTRAÇÃO COMPLETA - AI FITNESS COACH")
    print("=" * 80)
    
    # Cenários de teste
    cenarios = [
        {
            "nome": "João - Iniciante Hipertrofia",
            "dados": {
                "idade": 22,
                "peso": 68.0,
                "altura": 1.75,
                "periodicidade": 3,
                "objetivo": "hipertrofia"
            }
        },
        {
            "nome": "Maria - Emagrecimento Avançado", 
            "dados": {
                "idade": 32,
                "peso": 72.0,
                "altura": 1.68,
                "periodicidade": 5,
                "objetivo": "emagrecimento"
            }
        },
        {
            "nome": "Carlos - Força Experiente",
            "dados": {
                "idade": 28,
                "peso": 85.0,
                "altura": 1.82,
                "periodicidade": 4,
                "objetivo": "forca"
            }
        }
    ]
    
    resultados = []
    
    for i, cenario in enumerate(cenarios, 1):
        print(f"\n🧪 CENÁRIO {i}: {cenario['nome']}")
        print("-" * 50)
        
        # Mostrar dados do usuário
        dados = cenario['dados']
        print("👤 Perfil:")
        for campo, valor in dados.items():
            emoji = {"idade": "📅", "peso": "⚖️", "altura": "📏", 
                    "periodicidade": "📅", "objetivo": "🎯"}
            print(f"   {emoji.get(campo, '•')} {campo.title()}: {valor}")
        
        print(f"\n🔄 Executando pipeline...")
        
        # Executar sistema
        resultado = executar_fitness_coach(dados)
        
        if resultado:
            # Estatísticas rápidas
            print(f"✅ Sucesso!")
            print(f"   📊 IMC: {resultado['imc']} ({resultado['classificacao_imc']})")
            print(f"   🔥 Calorias/treino: {resultado['gasto_calorico']} kcal")
            print(f"   📋 Treinos gerados: {len(resultado['planilha_treinos'])}")
            
            # Mostrar primeiro treino como exemplo
            if resultado['planilha_treinos']:
                primeiro_treino = resultado['planilha_treinos'][0]
                print(f"   💪 Primeiro treino: {primeiro_treino['nome']}")
                print(f"   🏋️‍♂️ Exercícios: {len(primeiro_treino['exercicios'])}")
            
            resultados.append((cenario['nome'], resultado))
        else:
            print("❌ Erro na execução")
        
        print()
    
    # Resumo final
    print("🎊 DEMONSTRAÇÃO CONCLUÍDA!")
    print("=" * 80)
    print(f"✅ Cenários executados com sucesso: {len(resultados)}/{len(cenarios)}")
    
    if resultados:
        print("\n📊 COMPARAÇÃO DOS RESULTADOS:")
        print("-" * 40)
        
        for nome, resultado in resultados:
            dados = resultado['dados_limpos']
            print(f"{nome}:")
            print(f"   🎯 {dados['objetivo'].title()} | {dados['periodicidade']}x/semana")
            print(f"   📊 IMC {resultado['imc']} | {resultado['gasto_calorico']} kcal")
            print()
        
        # Oferecer salvamento do último resultado
        print("💾 SALVAMENTO:")
        ultimo_nome, ultimo_resultado = resultados[-1]
        print(f"Salvando planilha de '{ultimo_nome}'...")
        
        sucesso = salvar_planilha(
            ultimo_resultado, 
            f"planilha_{ultimo_nome.lower().replace(' ', '_').replace('-', '_')}"
        )
        
        if sucesso:
            print("✅ Arquivo salvo com sucesso!")
            print("\n📋 Para visualizar a planilha completa:")
            print(f"   exibir_planilha_avancada(resultados[-1][1])")
    
    return resultados

# Executar demonstração
print("🚀 Iniciando demonstração completa...")
print("⏱️ Isso pode levar alguns segundos...")
print()

demo_resultados = demonstracao_completa()

print("\n🎉 DEMONSTRAÇÃO FINALIZADA!")
print("💡 Agora você pode:")
print("   1. Executar exibir_planilha_avancada(demo_resultados[0][1]) para ver planilha completa")
print("   2. Executar gerar_relatorio_completo(demo_resultados[0][1]) para relatório detalhado") 
print("   3. Criar sua própria planilha com dados_usuario = coletar_dados_usuario()")

---

## 📚 **CONCLUSÃO E ARQUITETURA**

### ✅ **Funcionalidades Implementadas:**

1. **✅ Coleta e Validação de Dados** - Nó que recebe e valida entrada do usuário
2. **✅ Cálculos Matemáticos** - Ferramenta para IMC, calorias e distribuição de treinos  
3. **✅ Busca Web** - Integração com DuckDuckGo para pesquisa de melhores práticas
4. **✅ Sistema RAG** - Base de conhecimento fitness vetorizada para recuperação de informações
5. **✅ Geração de Conteúdo** - LLM para criar planilha de 12 treinos personalizados

### 🏗️ **Arquitetura LangGraph:**

```
📥 [Entrada] 
    ↓
🔍 [Nó Validação] → valida idade, peso, periodicidade, objetivo
    ↓
🧮 [Nó Cálculos] → calcula IMC, calorias, distribuição treinos
    ↓  
🌐 [Nó Busca Web] → pesquisa melhores práticas online
    ↓
🧠 [Nó RAG] → recupera exercícios e princípios da base conhecimento
    ↓
📝 [Nó Geração] → cria 12 treinos completos personalizados
    ↓
📤 [Saída] → planilha finalizada
```

### 🎯 **Diferenciais do Projeto:**

- **Personalização Completa** - Adapta-se a diferentes objetivos, idades e periodicidades
- **Base Científica** - Usa princípios estabelecidos de treinamento
- **Arquitetura Robusta** - LangGraph garante fluxo confiável e modular
- **Interface Amigável** - Coleta de dados intuitiva no Jupyter
- **Escalável** - Fácil de expandir com novos nós e funcionalidades

### 📈 **Possíveis Melhorias Futuras:**

1. **🤖 LLM Integration** - Usar Groq/OpenAI para descrições mais ricas dos exercícios
2. **🗄️ Banco de Dados** - Salvar histórico de usuários e progressões
3. **📊 Analytics** - Dashboard com métricas de desempenho
4. **🎥 Multimedia** - Links para vídeos demonstrativos dos exercícios
5. **📱 Mobile App** - Interface mobile para acompanhamento

---

**🏆 AI Fitness Coach - Trabalho Prático 03 Finalizado! 🏆**